In [96]:
import numpy as np
import pandas as pd
from sklearn import svm
import random
from sklearn.model_selection import train_test_split as ts
from sklearn.ensemble import RandomForestClassifier

In [156]:
traindf, labeldf, testdf = pd.read_csv('train.csv'), pd.read_csv('train_label.csv'), pd.read_csv('test.csv')

In [169]:
def encoding(dataframe, train=True):
    df = dataframe.copy()
    df.drop('ID', inplace=True, axis=1)
    df.drop('arrival_date_year', inplace=True, axis=1)
    if train:
        df.drop('reservation_status', inplace=True, axis=1)
        df.drop('reservation_status_date', inplace=True, axis=1)
    #df['arrival_date_year'] = df.arrival_date_year.map(lambda x:0 if x==2015 else(1 if x==2016 else 2))
    df['hotel'] = df.hotel.map(lambda x: 1 if x == 'City Hotel' else 0)
    df = pd.get_dummies(df, columns=['arrival_date_month'], dummy_na=False)
    df = pd.get_dummies(df, columns=['meal'], dummy_na=False)
    df['country'] = df.country.map(lambda x: 'Others' 
                               if x not in ['PRT','GBR','FRA','ESP','DEU','ITA','IRL','BRA','NLD','BEL','USA','CHE'] 
                               else x)
    df = pd.get_dummies(df, columns=['country'], dummy_na=False)
    df = pd.get_dummies(df, columns=['market_segment'], dummy_na=False)
    df = pd.get_dummies(df, columns=['distribution_channel'], dummy_na=False)
    
    #reserved_room_type & assigned_room_type
    possiblecat = list('ABCDEFGHIJKL')
    df['reserved_room_type'] = df['reserved_room_type'].astype('category', categories=possiblecat)
    df['assigned_room_type'] = df['assigned_room_type'].astype('category', categories=possiblecat)
    df = pd.get_dummies(df, columns=['reserved_room_type'], dummy_na=False)
    df = pd.get_dummies(df, columns=['assigned_room_type'], dummy_na=False)
    
    df = pd.get_dummies(df, columns=['deposit_type'], dummy_na=False)
    df['agent'] = df.agent.map(lambda x: 1 if x else 0)
    df['company'] = df.company.map(lambda x: 1 if x else 0)
    df = pd.get_dummies(df, columns=['customer_type'], dummy_na=False)
    
    #outcome variables
    if train:
        adr = df['adr']
        cancel = df['is_canceled']
    
        df.drop('adr', inplace=True, axis=1)
        df.drop('is_canceled', inplace=True, axis=1)
    
    df = df.fillna(0)
    
    if train:
        return df, adr, cancel
    else:
        return df

In [132]:
df,adr,cancel = encoding(traindf)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """Entry point for launching an IPython kernel.


In [135]:
X_train, X_test, y_train, y_test = ts(df,cancel,test_size=0.3,random_state=50)
model_rf = RandomForestClassifier(n_estimators = 100, oob_score = True, n_jobs = -1,random_state =50,
                                max_features = "auto",max_depth = 30)#min_samples_split=3)#,min_samples_leaf = leaf_option)
model_rf.fit(X_train, y_train)
model_rf.score(X_test,y_test)

0.8971595047341587

In [149]:
#輸出預測is_cancel,adr值的list
ypred_rf = model_rf.predict(df)
ypred_rfreg = model_rfreg.predict(df)

In [159]:
#把預測的adr,is_canceled加進training data中
traindf['adr_pred'] = ypred_rfreg
traindf['is_canceled_pred'] = ypred_rf

In [134]:
from sklearn.ensemble import RandomForestRegressor
#X_train, X_test, y_train, y_test = ts(df,adr,test_size=0.3)
X_train, X_test, y_train, y_test = ts(df,adr,test_size=0.3,random_state=50)
model_rfreg = RandomForestRegressor(200)
model_rfreg.fit(X_train,y_train)
model_rfreg.score(X_test,y_test)

0.816395838821959

In [26]:
from xgboost import XGBRegressor

modelXGBR = XGBRegressor(n_estimators=2000, learning_rate =0.01, max_depth=22, min_child_weight=11, gamma=0.2,
                     colsample_bytree=0.6, subsample=0.9, reg_alpha=0.1, reg_lambda=1e-05).fit(X_train, y_train)

print(modelXGBR.score(X_test,y_test))
ans = modelXGBR.predict(X_train)
modelXGBR.score(X_train,y_train)

0.8474491188039086


In [161]:
def fromdatatolabel(dataset,labelset):
    df = dataset.copy()
    label_df = labelset.copy()

    predictoutcome = []
    mlist = 'January,February,March,April,May,June,July,August,September,October,November,December'.split(',')
    for i in label_df['arrival_date']:
        y,m,d = i.split('-')
        subsetdf = traindf[(df['arrival_date_year'] == int(y)) & 
                           (df['arrival_date_month'] == mlist[int(m)-1]) &
                           (df['arrival_date_day_of_month'] == int(d))]

        subsetdf['new'] = (1-subsetdf['is_canceled_pred'])*(subsetdf['stays_in_weekend_nights']+subsetdf['stays_in_week_nights'])*subsetdf['adr_pred']

        predictoutcome.append(np.floor(subsetdf['new'].sum()/10000))
    return predictoutcome

In [162]:
labeldf['predlabel'] = fromdatatolabel(traindf, labeldf)
labeldf

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,arrival_date,label,predlabel
0,2015-07-01,2.0,2.0
1,2015-07-02,1.0,1.0
2,2015-07-03,1.0,1.0
3,2015-07-04,1.0,1.0
4,2015-07-05,1.0,1.0
5,2015-07-06,2.0,2.0
6,2015-07-07,1.0,1.0
7,2015-07-08,1.0,1.0
8,2015-07-09,1.0,0.0
9,2015-07-10,2.0,2.0


In [167]:
def ErrorFunction(labelset):
    cnt = 0
    mae = 0
    for index,value in labelset.iterrows():
        if(value['label']!=value['predlabel']):
            cnt+=1
            mae += abs(value['label']-value['predlabel'])
    print('0/1 error: ',cnt/len(labelset))
    print('MAE: ',mae/len(labelset))

In [168]:
ErrorFunction(labeldf)

0/1 error:  0.1234375
MAE:  0.1234375
